For the tabular portion of the multimodal probabilistic model fusion, a novel boosting method known as NGBoost is used. NGBoost is a boosting method that uses a series of weak tree-based learner, where it returns a predictive probability distribution instead of a point estimate, which is done by minimizing the negative log-likelihood.

In [10]:
import numpy as np
import pandas as pd
from pathlib import Path
from ngboost import NGBRegressor
from ngboost.distns import Normal
from sklearn.model_selection import train_test_split

For train-test split, it is done by unique patient IDs, as each row in the CSV is not a unique entry, but a series of rows for each patient ID is. Then, the corresponding y (outcome) only is the FVC value in the last recorded week for each patient.

In [ ]:
BASE_DIR = Path().resolve()
csv_path = BASE_DIR.parent / "data" / "train.csv"

df = pd.read_csv(csv_path)

ids = df['Patient'].unique()
train_ids, val_ids = train_test_split(ids, test_size=0.2, random_state=42)

train_df = df[df['Patient'].isin(train_ids)].reset_index(drop=True)
val_df   = df[df['Patient'].isin(val_ids)].reset_index(drop=True)